In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=1f74d6d67ccea749897e8072fcac3cfc07e24da5ef9eef245c65561a63c38384
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.06 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 18.26 , C = 956.14 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_13_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=928565, Tue Apr 14 11:43:22 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 7.502570036667099e+03 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 7.277543612175044e+03 1.0e+00 2.35e+00  2e+00  2e+00 0:00.1
    3     51 6.985587429513020e+03 1.0e+00 2.30e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8032.65115739] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7665.46533018] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   89   1513 3.110596889846873e+03 1.3e+00 9.99e-01  1e+00  1e+00 0:03.1
  100   1700 2.668468550756894e+03 1.3e+00 8.07e-01  8e-01  8e-01 0:03.5
  200   3400 1.803815954247553e+03 1.3e+00 1.30e-01  1e-01  1e-01 0:07.0
  300   5100 1.777033753498337e+03 1.4e+00 2.06e-02  2e-02  2e-02 0:10.4
  400   6800 1.776470264751405e+03 1.4e+00 3.10e-03  3e-03  3e-03 0:13.8
  500   8500 1.776454078563194e+03 1.4e+00 4.74e-04  4e-04  5e-04 0:17.2
  600  10200 1.776453769105438e+03 1.4e+00 7.97e-05  7e-05  8e-05 0:20.6
  700  11900 1.776453757525550e+03 1.4e+00 1.18e-05  1e-05  1e-05 0:24.0
  800  13600 1.776453757304452e+03 1.4e+00 2.02e-06  2e-06  2e-06 0:27.4
  900  15300 1.776453757297648e+03 1.4e+00 3.53e-07  3e-07  3e-07 0:30.8
 1000  17000 1.776453757297456e+03 1.4e+00 6.62e-08  6e-08  6e-08 0:34.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 1.776453757297452e+03 1.5e+00 2.98e-08  3e-08  3e-08 0:37.6
 1181  20077 1.776453757297450e+03 1.5e+00 2.07e-08 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7742.34580835] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=7742.345808, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=7742.345808, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

    3     51 6.575390657937543e+03 1.0e+00 2.29e+00  2e+00  2e+00 0:00.1
   80   1360 2.670640511506881e+03 1.3e+00 1.25e+00  1e+00  1e+00 0:03.1
  100   1700 2.413310114361515e+03 1.3e+00 9.59e-01  9e-01  1e+00 0:03.9
  200   3400 1.867818092530646e+03 1.3e+00 2.76e-01  3e-01  3e-01 0:08.0
  300   5100 1.801124563334147e+03 1.4e+00 9.06e-02  8e-02  9e-02 0:12.0
  400   6800 1.789049948564729e+03 1.5e+00 3.98e-02  4e-02  4e-02 0:16.1
  500   8500 1.787118810328388e+03 1.7e+00 1.61e-02  1e-02  2e-02 0:20.0
  600  10200 1.786769399220531e+03 1.8e+00 7.72e-03  6e-03  9e-03 0:24.0
  700  11900 1.786678971095940e+03 2.0e+00 3.61e-03  3e-03  4e-03 0:28.2
  800  13600 1.786656263446711e+03 2.3e+00 1.85e-03  1e-03  2e-03 0:32.2
  900  15300 1.786651068936386e+03 2.6e+00 8.57e-04  7e-04  1e-03 0:36.2
 1000  17000 1.786649929377525e+03 2.9e+00 4.81e-04  4e-04  7e-04 0:40.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 1.786649619066348e+03 3.2e+00 2.39e-04 

...

    3     51 6.194582858180185e+03 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1
   80   1360 2.701361438345192e+03 1.3e+00 1.18e+00  1e+00  1e+00 0:03.1
  100   1700 2.410681574440482e+03 1.3e+00 8.97e-01  9e-01  9e-01 0:03.9
  200   3400 1.866884474068154e+03 1.3e+00 2.44e-01  2e-01  3e-01 0:07.9
  300   5100 1.805053676768532e+03 1.4e+00 8.91e-02  8e-02  9e-02 0:11.8
  400   6800 1.791330991577854e+03 1.6e+00 4.75e-02  4e-02  5e-02 0:15.9
  500   8500 1.787484896999357e+03 1.8e+00 2.03e-02  2e-02  2e-02 0:19.9
  600  10200 1.786832451126857e+03 2.2e+00 1.13e-02  9e-03  1e-02 0:23.8
  700  11900 1.786689310391500e+03 2.4e+00 4.17e-03  3e-03  5e-03 0:27.9
  800  13600 1.786658245027041e+03 2.6e+00 2.00e-03  2e-03  2e-03 0:31.9
  900  15300 1.786651843718719e+03 2.7e+00 1.06e-03  8e-04  1e-03 0:36.0
 1000  17000 1.786650337683952e+03 2.9e+00 4.63e-04  3e-04  6e-04 0:40.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 1.786649888757470e+03 3.1e+00 3.13e-04 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7313.86833266] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6799.30568519] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   91   1547 3.270472314022293e+03 1.3e+00 1.06e+00  1e+00  1e+00 0:03.1
  100   1700 2.922161189742781e+03 1.3e+00 8.89e-01  9e-01  9e-01 0:03.4
  200   3400 1.810985922870526e+03 1.3e+00 1.45e-01  1e-01  1e-01 0:06.9
  300   5100 1.777411642856490e+03 1.3e+00 2.32e-02  2e-02  2e-02 0:10.3
  400   6800 1.776483365256967e+03 1.3e+00 4.07e-03  4e-03  4e-03 0:13.7
  500   8500 1.776454508723872e+03 1.3e+00 6.66e-04  6e-04  6e-04 0:17.0
  600  10200 1.776453773728085e+03 1.3e+00 1.11e-04  1e-04  1e-04 0:20.5
  700  11900 1.776453757844517e+03 1.3e+00 1.82e-05  2e-05  2e-05 0:23.8
  800  13600 1.776453757310262e+03 1.3e+00 2.81e-06  3e-06  3e-06 0:27.2
  900  15300 1.776453757297781e+03 1.4e+00 4.79e-07  4e-07  5e-07 0:30.7
 1000  17000 1.776453757297462e+03 1.4e+00 8.68e-08  8e-08  8e-08 0:34.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 1.776453757297453e+03 1.4e+00 3.24e-08  3e-08  3e-08 0:37.5
 1189  20213 1.776453757297452e+03 1.5e+00 2.02e-08 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7742.34580835] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=7742.345808, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=7742.345808, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

    3     51 6.313578624987567e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1
   79   1343 2.653412506417680e+03 1.3e+00 1.15e+00  1e+00  1e+00 0:03.1
  100   1700 2.382266573971652e+03 1.3e+00 8.86e-01  9e-01  9e-01 0:03.9
  200   3400 1.871807088515786e+03 1.3e+00 2.11e-01  2e-01  2e-01 0:07.9
  300   5100 1.800687191812679e+03 1.5e+00 9.66e-02  9e-02  1e-01 0:11.9
  400   6800 1.788975700093275e+03 1.6e+00 3.60e-02  3e-02  4e-02 0:15.9
  500   8500 1.787037459423477e+03 1.7e+00 1.69e-02  1e-02  2e-02 0:19.9
  600  10200 1.786730265757535e+03 1.8e+00 6.63e-03  6e-03  7e-03 0:23.9
  700  11900 1.786664181109802e+03 2.0e+00 3.38e-03  3e-03  4e-03 0:27.8
  800  13600 1.786652481802893e+03 2.3e+00 1.13e-03  9e-04  1e-03 0:31.7
  900  15300 1.786650257593285e+03 2.7e+00 6.01e-04  5e-04  8e-04 0:35.7
 1000  17000 1.786649818860581e+03 3.0e+00 3.68e-04  3e-04  6e-04 0:39.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 1.786649617114440e+03 3.2e+00 1.86e-04 

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F13_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:100])
    SVM_Fun [i] = F13(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F13(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1819.50817179]), count=array([1])) 1.574572247937148e-05
SVM
ModeResult(mode=array([6933.83194912]), count=array([1])) 459.13040842032916
ELN
ModeResult(mode=array([4958.57897367]), count=array([1])) 2.5853828191179307e-05
